In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from statistics import mean

In [4]:
#Функция получения имен типов
def get_type_names(nested):
    """Метод принимает колонку со словарем вложенных элементов и вытаскивает имена через точку с запятой

    Parameters
    ----------
    nested
        Колонка со словарем вложенных элементов

    Returns
    -------
        Строка с перечнем типов через запятую
    """
    res = []
    for item in nested:
        type_name = item['TypeName']
        res.append(type_name)
    return ';'.join(res)

#Функция получения средних координат
def get_mean_coord(nested):
    """Метод расчета средних координат X,Y,Z - записывается через запятую

    Parameters
    ----------
    nested
        Колонка с координатами

    Returns
    -------
        X;Y;Z средние
    """
    x_arr = []
    y_arr = []
    z_arr = []
    res = []
    for item in nested:
        for i in range(0,len(item)-1):
            x = item['Points'][i]['X']
            x_arr.append(x)
            y = item['Points'][i]['Y']
            y_arr.append(y)
            z = item['Points'][i]['Z']
            z_arr.append(z)
    if len(x_arr) > 0:
        res.append(str(mean(x_arr)))
    else:
        res.append('0.0')
    if len(y_arr) > 0:
        res.append(str(mean(y_arr)))
    else:
        res.append('0.0')
    if len(z_arr) > 0:
        res.append(str(mean(z_arr)))
    else:
        res.append('0.0')
    return ';'.join(res)

#Функция конвертации в float
def convertToDouble(value):
    """Метод перевода числовых колонок в float

    Parameters
    ----------
    value
        Колонка

    Returns
    -------
        Double, если успех, value - если неуспех
    """
    try:
        return value.astype(float)
    except:
            return value

In [5]:
#Загрузка данных
# df = pd.read_json(r'D:\Khabarov\Репозиторий\panels_analysis\PanelDTOs.json')
df = pd.read_json(r'D:\Khabarov\Репозиторий\panels_analysis\Data\Проект1_PrefabPanelGroup.json')
len(df)

25

In [6]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType, StringTensorType, BooleanTensorType
from skl2onnx import update_registered_converter

#Класс-трансформер для преобразования координат
class CoorMeanTransformer(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        df = X.copy()

        #Создание колонку с информацией по среднему расположению полостей
        df['Cavities_Coord'] = df['Cavities'].apply(get_mean_coord)
        df['Cavities_X'] = df['Cavities_Coord'].str.split(';').str[0]
        df['Cavities_Y'] = df['Cavities_Coord'].str.split(';').str[1]
        df['Cavities_Z'] = df['Cavities_Coord'].str.split(';').str[2]

        #Создание колонку с информацией по среднему расположению закладных
        df['Embeddeds_Coord'] = df['Embeddeds'].apply(get_mean_coord)
        df['Embeddeds_X'] = df['Embeddeds_Coord'].str.split(';').str[0]
        df['Embeddeds_Y'] = df['Embeddeds_Coord'].str.split(';').str[1]
        df['Embeddeds_Z'] = df['Embeddeds_Coord'].str.split(';').str[2]

        #Создание колонку с информацией по среднему расположению сетей
        df['UtilNetworks_Coord'] = df['UtilNetworks'].apply(get_mean_coord)
        df['UtilNetworks_X'] = df['UtilNetworks_Coord'].str.split(';').str[0]
        df['UtilNetworks_Y'] = df['UtilNetworks_Coord'].str.split(';').str[1]
        df['UtilNetworks_Z'] = df['UtilNetworks_Coord'].str.split(';').str[2]
        return df

#Метод для передачи в регистратор
def coor_mean_transformer(scope,operator,container):
    input_name = operator.input_full_names
    output_name = operator.output_full_names
    container.add_node("Identity",input_name,output_name)

#Метод для определения выходного размера (для ONNX)
def shape_coor_mean_transformer(operator):
    """ Определение размеров данных для ONNX """
    new_cols_count = 9
    input_shape = operator.inputs[0].type.shape  # Размер входного массива
    operator.outputs[0].type = FloatTensorType([None, input_shape[1] + new_cols_count])  # +9 новых колонок (X, Y, Z)

#Регистрируем пользовательский трансформер в `skl2onnx`
update_registered_converter(
    CoorMeanTransformer,  # Класс трансформера
    "CoorMeanTransformer",  # Название в ONNX
    shape_coor_mean_transformer,
    coor_mean_transformer,
)


In [7]:
#Класс-трансформер для разбивки информации по вложенным элементам
class ItemsTypesTransformer(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        df = X.copy()

        #Создание dummy с информацией по полостям
        df['Cavities_Types'] = df['Cavities'].apply(get_type_names)
        #Создание dummy с информацией по закладным деталям
        df['Embeddeds_Types'] = df['Embeddeds'].apply(get_type_names)
        #Создание dummy с информацией по инженерке
        df['UtilNetworks_Types'] = df['UtilNetworks'].apply(get_type_names)

        #Добавление кол-ва
        df['Cavities_ct'] = np.where(df['Cavities_Types'] == '',0, df['Cavities_Types'].str.split(';').str.len())
        df['Embeddeds_ct'] = np.where(df['Embeddeds_Types'] == '',0,df['Embeddeds_Types'].str.split(';').str.len())
        df['UtilNetworks_ct'] = np.where(df['UtilNetworks_Types'] == '',0,df['UtilNetworks_Types'].str.split(';').str.len())
        return df

#Метод для передачи в регистратор
def items_types_transformer(scope,operator,container):
    input_name = operator.input_full_names
    output_name = operator.output_full_names
    container.add_node("Identity",input_name,output_name)

#Метод для определения выходного размера (для ONNX)
def shape_items_types_transformer(operator):
    """ Определение размеров данных для ONNX """
    new_cols_count = 6
    input_shape = operator.inputs[0].type.shape  # Размер входного массива
    operator.outputs[0].type = FloatTensorType([None, input_shape[1] + new_cols_count])  # +9 новых колонок (X, Y, Z)

#Регистрируем пользовательский трансформер в `skl2onnx`
update_registered_converter(
    ItemsTypesTransformer,  # Класс трансформера
    "ItemsTypesTransformer",  # Название в ONNX
    shape_items_types_transformer,
    items_types_transformer,
)

In [8]:
#Класс-трансформер для добавления dummy переменных по вложенным
class ItemsDummiesTransformer(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        df = X.copy()

        #Создание dummy с информацией по полостям
        cavities_types_dummies = df['Cavities_Types'].str.get_dummies(sep=';')
        #Создание dummy с информацией по закладным деталям
        embeddeds_types_dummies = df['Embeddeds_Types'].str.get_dummies(sep=';')
        #Создание dummy с информацией по инженерке
        networks_types_dummies = df['UtilNetworks_Types'].str.get_dummies(sep=';')

        #Добавление
        df_with_cav_dummy = pd.concat([df,cavities_types_dummies],axis=1)
        df_with_emb_dummy = pd.concat([df_with_cav_dummy,embeddeds_types_dummies],axis=1)
        df_final = pd.concat([df_with_emb_dummy,networks_types_dummies],axis=1)

        return df_final
    
#Метод для передачи в регистратор
def items_dummies_transformer(scope,operator,container):
    input_name = operator.input_full_names
    output_name = operator.output_full_names
    container.add_node("Identity",input_name,output_name)

#Метод для определения выходного размера (для ONNX)
def shape_items_dummies_transformer(operator):
    """ Определение размеров данных для ONNX """
    new_cols_count = 100
    input_shape = operator.inputs[0].type.shape  # Размер входного массива
    operator.outputs[0].type = FloatTensorType([None, input_shape[1] + new_cols_count])

#Регистрируем пользовательский трансформер в `skl2onnx`
update_registered_converter(
    ItemsDummiesTransformer,  # Класс трансформера
    "ItemsDummiesTransformer",  # Название в ONNX
    shape_items_dummies_transformer,
    items_dummies_transformer,
)

In [9]:
#Класс-трансформер для приведения типов
class ConvertToDoubleTransformer(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        df = X.copy()
        df = df.apply(convertToDouble)
        return df
    
#Метод для передачи в регистратор
def сonvert_to_double_transformer(scope,operator,container):
    input_name = operator.input_full_names
    output_name = operator.output_full_names
    container.add_node("Identity",input_name,output_name)

#Метод для определения выходного размера (для ONNX)
def shape_сonvert_to_double_transformer(operator):
    """ Определение размеров данных для ONNX """
    new_cols_count = 0
    input_shape = operator.inputs[0].type.shape  # Размер входного массива
    operator.outputs[0].type = FloatTensorType([None, input_shape[1] + new_cols_count])

#Регистрируем пользовательский трансформер в `skl2onnx`
update_registered_converter(
    ConvertToDoubleTransformer,  # Класс трансформера
    "ConvertToDoubleTransformer",  # Название в ONNX
    shape_сonvert_to_double_transformer,
    сonvert_to_double_transformer,
)


In [10]:
#Класс-трансформер для подготовки данных на вход модели
class RemoveOtherColumns(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        df = X.copy()
        df_general = df.drop(labels=['ProjectName','Count','Name'
                      ,'Number','Cavities','Embeddeds'
                      #,'Name','ProjectName','Count','Number'
                      #,'Cavities_Coord','Embeddeds_Coord','UtilNetworks_Coord'
                      #,'Type','FireResist','Length','Height','Width','Thickness'
                      ,'UtilNetworks','Components','Cavities_Coord'
                      ,'Cavities_Types','Embeddeds_Types','UtilNetworks_Types'
                      ,'Embeddeds_Coord','UtilNetworks_Coord'],axis=1)
        return df_general

In [11]:
#Импортирование скейлера и эстимейтора
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [12]:
#Класс-трансформер для заполнения пустых значений
class FillNanData(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        transformed_df = X.copy()
        transformed_df.loc[:,transformed_df.select_dtypes(include=['object']).columns] = transformed_df.select_dtypes(include=['object']).replace("", "Не заполнено").fillna('Не заполнено')
        transformed_df.loc[:,transformed_df.select_dtypes(exclude=['object']).columns] = transformed_df.select_dtypes(exclude=['object']).fillna(0)
        return transformed_df


In [13]:
#Класс-трансформер для масштабирования и обучения по группам
class ScaleAndFitData(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        transformed_df = X.copy()
        scaler = StandardScaler()
        dbscan = DBSCAN(eps=0.01,min_samples=2)
        #Колонки для группировки
        cols_to_group = ['Type','FireResist','Length'
                               ,'Height','Width','Thickness'
                               ,'IsPipeContains'
                               ,'Cavities_ct','Embeddeds_ct','UtilNetworks_ct']
        #Колонки для масштабирования
        cols_to_scale = list(transformed_df.drop(labels=cols_to_group,axis=1).columns)
        cols_to_scale

        #Масштабируем нужные признаки, группируя по нужным колонкам
        scaled_df = transformed_df.groupby(cols_to_group)[cols_to_scale].apply(lambda x: pd.DataFrame(scaler.fit_transform(x), index=x.index, columns=x.columns))
        #Кластеризуем данные в группах
        clusters_df = scaled_df.groupby(cols_to_group)[cols_to_scale].apply(lambda x: pd.DataFrame(dbscan.fit_predict(x), index=x.index,columns=['Cluster']))

        #Возвращаем кластеры
        clusters = clusters_df.reset_index(drop=True)['Cluster']
        return clusters

In [14]:
#Тест операции
pipeline = Pipeline([('CoorMeanTransformer',CoorMeanTransformer())
                     ,('ItemsTypesTransformer',ItemsTypesTransformer())
                     ,('ItemsDummiesTransformer',ItemsDummiesTransformer())
                     ,('ConvertToDoubleTransformer',ConvertToDoubleTransformer())
                    ,('FillNaData',FillNanData())
                    ,('RemoveOtherColumns',RemoveOtherColumns())
                    ,('ScaleAndFitData',ScaleAndFitData())
                     ])
clusters = pipeline.fit_transform(df)
clusters




0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    2
21    2
22    2
23    3
24    3
Name: Cluster, dtype: int64

In [15]:
#Добавление к исходному датафрейму
df['Cluster'] = clusters
df

,Name,ProjectName,Count,Type,FireResist,Length,Height,Width,Thickness,Volume,IsPipeContains,Number,NumberNetwork,Components,Cavities,Embeddeds,UtilNetworks,Cluster
0,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.14,False,1,1,NaN,[],"[{'TypeName': 'ЗД21', 'Points': [{'Z': 0.42650...",[],0
1,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
2,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
3,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
4,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
5,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
6,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
7,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
8,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1
9,ПБК-343.157.18-1,Проект1,1,ПБК,,3435,0,1570,180,1.06,False,1,1,NaN,[],"[{'TypeName': 'ЗД19', 'Points': [{'Z': 0.73818...",[],1


In [16]:
#Сохранение модели
import joblib
joblib.dump(pipeline, r'D:\Khabarov\Репозиторий\panels_analysis\Data\Model\pipeline.pkl')
print('Модель сохранена')

Модель сохранена


In [18]:
#Использование модели
loaded_pipeline = joblib.load(r'D:\Khabarov\Репозиторий\panels_analysis\Data\Model\pipeline.pkl')
prediction = loaded_pipeline.fit_transform(df)
prediction

0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    2
21    2
22    2
23    3
24    3
Name: Cluster, dtype: int64

In [ ]:
# #Типы колонок
# #  0   Name            25 non-null     object 
# #  1   ProjectName     25 non-null     object 
# #  2   Count           25 non-null     int64  
# #  3   Type            25 non-null     object 
# #  4   FireResist      25 non-null     object 
# #  5   Length          25 non-null     int64  
# #  6   Height          25 non-null     int64  
# #  7   Width           25 non-null     int64  
# #  8   Thickness       25 non-null     int64  
# #  9   Volume          25 non-null     float64
# #  10  IsPipeContains  25 non-null     bool   
# #  11  Number          25 non-null     int64  
# #  12  NumberNetwork   25 non-null     int64  
# #  13  Components      0 non-null      float64
# #  14  Cavities        25 non-null     object 
# #  15  Embeddeds       25 non-null     object 
# #  16  UtilNetworks    25 non-null     object
# # df.info()

# #Определение типов входных данных
# import onnxruntime as rt
# from skl2onnx.common.data_types import FloatTensorType, StringTensorType, BooleanTensorType, Int64TensorType, DoubleTensorType
# from skl2onnx import convert_sklearn
# initial_types = [('index',FloatTensorType([None,1]))
#                 ,('Name',StringTensorType([None,1]))
#                 ,('ProjectName',StringTensorType([None,1]))
#                 ,('Count',FloatTensorType([None,1]))
#                 ,('Type',StringTensorType([None,1]))
#                 ,('FireResist',StringTensorType([None,1]))
#                 ,('Length',FloatTensorType([None,1]))
#                 ,('Height',FloatTensorType([None,1]))
#                 ,('Width',FloatTensorType([None,1]))
#                 ,('Thickness',FloatTensorType([None,1]))
#                 ,('Volume',FloatTensorType([None,1]))
#                 ,('IsPipeContains',BooleanTensorType([None,1]))
#                 ,('Number',FloatTensorType([None,1]))
#                 ,('NumberNetwork',FloatTensorType([None,1]))
#                 ,('Components',FloatTensorType([None,1]))
#                 ,('Cavities',StringTensorType([None,1]))
#                 ,('Embeddeds',StringTensorType([None,1]))
#                 ,('UtilNetworks',StringTensorType([None,1]))
#                 ]
# #Конвертация в ONNX
# onnx_model = convert_sklearn(pipeline, initial_types=initial_types)

# #Сохраняем в файл
# with open(r'D:\Khabarov\Репозиторий\panels_analysis\pipeline.onnx', "wb") as f:
#     f.write(onnx_model.SerializeToString())

# print("Модель сохранена в ONNX!")

Модель сохранена в ONNX!


In [ ]:
# import onnxruntime as rt

# #Загружаем ONNX Pipeline
# session = rt.InferenceSession(r'D:\Khabarov\Репозиторий\panels_analysis\pipeline.onnx')

# X_test = {"index": np.array([[0]], dtype=np.float32),
#     "Name": np.array([["PipeX"]], dtype=str),
#     "ProjectName": np.array([["Project1"]], dtype=str),
#     "Count": np.array([[10]], dtype=np.float32),
#     "Type": np.array([["Steel"]], dtype=str),
#     "FireResist": np.array([[30]], dtype=np.float32),
#     "Length": np.array([[200]], dtype=np.float32),
#     "Height": np.array([[50]], dtype=np.float32),
#     "Width": np.array([[40]], dtype=np.float32),
#     "Thickness": np.array([[2]], dtype=np.float32),
#     "Volume": np.array([[500.0]], dtype=np.float32),
#     "IsPipeContains": np.array([[1]], dtype=bool),  # Булевый признак
#     "Number": np.array([[12345]], dtype=np.float32),
#     "NumberNetwork": np.array([[567]], dtype=np.float32),
#     "Components": np.array([["ComponentA"]], dtype=str),
#     "Cavities": np.array([["Cavity1"]], dtype=str),
#     "Embeddeds": np.array([["EmbeddedB"]], dtype=str),
#     "UtilNetworks": np.array([["NetworkC"]], dtype=str)
# }

# inp = df.reset_index().loc[0].to_dict()
# # predictions = session.run(None,inp)
# # prediction


AttributeError: 'dict' object has no attribute 'dtype'